In [1]:
# -*- coding: utf-8 -*-
from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd
import json



In [2]:
#Функция проходит по всем возможным страницам и собирает вакансии
def parser_hh(vacancy):

    vacancy_date = []

    params = {
        'text': vacancy,
        'page': ''
    }

    headers = {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:69.0) Gecko/20100101 Firefox/69.0'
    }

    link = 'https://hh.ru/search/vacancy'

    html = requests.get(link, params=params, headers=headers)

    if html.ok:
        parsed_html = bs(html.text,'html.parser')

        page_block = parsed_html.find('div', {'data-qa': 'pager-block'})
        if not page_block:
            last_page = 1
        else:
            last_page = page_block.find_all('a', {'class': 'bloko-button'})
            last_page = int(last_page[len(last_page)-2].find('span').getText())

    for page in range(0, last_page):
        params['page'] = page
        html = requests.get(link, params=params, headers=headers)

        if html.ok:
            parsed_html = bs(html.text,'html.parser')

            vacancy_items = parsed_html.find_all('div', {'class': 'vacancy-serp-item'})
            #Здесь вызывается функция и она парсит одну страницу, потом следующую и т.д
            for item in vacancy_items:
                vacancy_date.append(parser_item_hh(item))

    return vacancy_date

In [3]:
#Функция парсит вакансии и отбирает нужные данные/атрибуты, 
#в качестве входного параметра берутся вакансии одной страницы
def parser_item_hh(item):

    vacancy_dict = {}

    vacancy_name = item.find('a', {'data-qa': 'vacancy-serp__vacancy-title'}) \
            .getText().replace(u'\xa0', u' ')

    vacancy_dict['vacancy_name'] = vacancy_name

    company_name = item.find('a', {'data-qa': 'vacancy-serp__vacancy-employer'})

    if company_name:
        company_name = company_name.getText().replace(u'\xa0', u' ')
    vacancy_dict['company_name'] = company_name

    city = item.find('span', {'data-qa': 'vacancy-serp__vacancy-address'}) \
        .getText() \
        .split(', ')[0]

    vacancy_dict['city'] = city

        # salary
    salary = item.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'})
    if not salary:
        salary_min = None
        salary_max = None
        salary_currency = None
    else:
        salary = salary.getText().replace(u'\u202f', u'')
        salary = salary.replace('–', '')
        salary = re.split(r'\s', salary)

        if salary[0] == 'до':
            salary_min = None
            salary_max = int(salary[1])
        elif salary[0] == 'от':
            salary_min = int(salary[1])
            salary_max = None
        else:
            salary_min = int(salary[0])
            salary_max = int(salary[2])

        salary_currency = salary[len(salary)-1]

    vacancy_dict['salary_min'] = salary_min
    vacancy_dict['salary_max'] = salary_max
    vacancy_dict['salary_currency'] = salary_currency

    vacancy_link = item.find('a', {'data-qa': 'vacancy-serp__vacancy-title'})['href']

    vacancy_dict['vacancy_link'] = vacancy_link
    vacancy_dict['site'] = 'hh.ru'

    return vacancy_dict

In [11]:
def parser_superjob(vacancy):
    vacancy_date = []

    params = {
        'keywords': vacancy, \
        'profession_only': '1', \
        'geo[c][0]': '15', \
        'geo[c][1]': '1', \
        'geo[c][2]': '9', \
        'page': ''
    }

    headers = {'User-Agent':
                   'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36'
               }

    link = 'https://www.superjob.ru/vacancy/search/'

    html = requests.get(link, params=params, headers=headers)

    if html.ok:
        parsed_html = bs(html.text, 'html.parser')

        page_block = parsed_html.find('a', {'class': 'f-test-button-dalshe'})
    if not page_block:
        last_page = 1
    else:
        last_page = int(page_block.previous_sibling.find_all('span')[-2].getText())

    for page in range(1, last_page + 1):
        params['page'] = page
        html = requests.get(link, params=params, headers=headers)

        if html.ok:
            parsed_html = bs(html.text, 'html.parser')
            vacancy_items = parsed_html.find_all('div', {'class': 'f-test-vacancy-item'})

            for item in vacancy_items:
                vacancy_date.append(parser_item_superjob(item))

    return vacancy_date

In [5]:
def parser_item_superjob(item):
    vacancy_dict = {}

    # vacancy_name
    vacancy_name = item.find_all('a')
    if len(vacancy_name) > 1:
        vacancy_name = vacancy_name[-2].getText()
    else:
        vacancy_name = vacancy_name[0].getText()

    # company_name
    company_name = item.find('span', {'class': 'f-test-text-vacancy-item-company-name'})

    if company_name:
        company_name = company_name.getText()
    else:
       company_name = None


    # city
    company_location = item.find('span', {'class': 'f-test-text-company-item-location'}) \
        .findChildren()[2] \
        .getText() \
        .split(',')


    # salary
    salary = item.find('span', {'class': 'f-test-text-company-item-salary'}) \
        .findChildren()
    if not salary:
        salary_min = None
        salary_max = None
        salary_currency = None
    else:
        salary_elems = salary[0].getText().replace(u'\xa0', u' ').split(' ')
        salary_currency = salary_elems[-1]
        is_check_salary = item.find('span', {'class': 'f-test-text-company-item-salary'}).getText()
        if is_check_salary != 'По договорённости':
            is_check_salary = is_check_salary.replace(u'\xa0', u' ').split(' ', 1)
            if is_check_salary[0] == 'до' or len(salary) == 2:
                if salary_elems[1].isdigit() and salary_elems[2].isdigit():
                    salary_min = None
                    salary_max = int(salary_elems[1] + salary_elems[2])
                elif salary_elems[1].isdigit() and not salary_elems[2].isdigit():
                    salary_max = int(salary_elems[1])
            elif is_check_salary[0] == 'от':
                if salary_elems[1].isdigit() and salary_elems[2].isdigit():
                    salary_min = int(salary_elems[1] + salary_elems[2])
                elif salary_elems[1].isdigit() and not salary_elems[2].isdigit():
                    salary_min = int(salary_elems[1])
                salary_max = None
            else:
                if len(salary_elems) > 3:
                    if salary_elems[1].isdigit():
                        salary_min = int(salary_elems[0] + salary_elems[1])
                    else:
                        salary_min = int(salary_elems[0])
                    if salary_elems[-2].isdigit() and salary_elems[-3].isdigit():
                        salary_max = int(salary_elems[-3] + salary_elems[-2])
                    else:
                        salary_max = int(salary_elems[-2])
                else:
                    if salary_elems[1].isdigit():
                        salary_min = int(salary_elems[0] + salary_elems[1])
                        salary_max = int(salary_elems[0] + salary_elems[1])
                    else:
                        salary_min = int(salary_elems[0])
                        salary_max = int(salary_elems[0])
        else:
            salary_min = None
            salary_max = None
            salary_currency = None


    # link
    vacancy_link = item.find_all('a')

    if len(vacancy_link) > 1:
        vacancy_link = vacancy_link[-2]['href']
    else:
        vacancy_link = vacancy_link[0]['href']

    if company_name != vacancy_name:
        vacancy_dict['vacancy_name'] = vacancy_name
        vacancy_dict['company_name'] = company_name
        vacancy_dict['city'] = company_location[0]
        vacancy_dict['salary_min'] = salary_min
        vacancy_dict['salary_max'] = salary_max
        vacancy_dict['salary_currency'] = salary_currency
        vacancy_dict['vacancy_link'] = f'https://www.superjob.ru{vacancy_link}'
        vacancy_dict['site'] = 'www.superjob.ru'
        return vacancy_dict

In [6]:
def parser_vacancy(vacancy):

    vacancy_data = []
    vacancy_data.extend(parser_hh(vacancy))
    vacancy_data.extend(parser_superjob(vacancy))
    vacancy_data = filter(None, vacancy_data)

    df = pd.DataFrame(vacancy_data)

    return df

In [7]:
df = parser_vacancy('python')
result = df.to_json('collected_data.json', orient='records', force_ascii=False)

In [8]:
df



,vacancy_name,company_name,city,salary_min,salary_max,salary_currency,vacancy_link,site
0,Backend разработчик на Python,Gigsty,Санкт-Петербург,250000.0,300000.0,руб.,https://voronezh.hh.ru/vacancy/47103960?from=v...,hh.ru
1,Senior Python developer,КА Натальи Зотовой,Москва,NaN,5500.0,USD,https://voronezh.hh.ru/vacancy/47139730?from=v...,hh.ru
2,Python-разработчик,Сбер. IT,Москва,NaN,NaN,None,https://voronezh.hh.ru/vacancy/47084105?from=v...,hh.ru
3,Junior Data Scientist,ООО IHS Global,Минск,NaN,NaN,None,https://voronezh.hh.ru/vacancy/47146916?from=v...,hh.ru
4,Backend разработчик (Python),ИП CyberTech.kz,Алматы,300000.0,600000.0,KZT,https://voronezh.hh.ru/vacancy/47039260?from=v...,hh.ru
...,...,...,...,...,...,...,...,...
810,Инженер-программист Python,ФГБУ ФЦМН ФМБА России,Москва,80000.0,NaN,руб.,https://www.superjob.ru/vakansii/inzhener-prog...,www.superjob.ru
811,Python developer,КонсалтПро,Москва,160000.0,200000.0,руб.,https://www.superjob.ru/vakansii/python-develo...,www.superjob.ru
812,"Middle, senior (C++, Python), инженер",FITTIN,Воронеж,80000.0,100000.0,руб.,https://www.superjob.ru/vakansii/middle-384209...,www.superjob.ru
813,Backend-разработчик Python,Турчанинов,Тула,40000.0,NaN,руб.,https://www.superjob.ru/vakansii/backend-razra...,www.superjob.ru
